In [1]:
from datetime import datetime, timedelta
from pathlib import Path
import numpy as np
import pandas as pd
import re
from IPython.display import HTML
from bs4 import BeautifulSoup
import AMBRA_Backups
from AMBRA_Backups.redcap_funcs import comp_schema_cap_db
import AMBRA_Utils

db_name = 'TESTED'
db = AMBRA_Backups.database.Database(db_name)
project_name = 'TESTED Data Collection'
project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name)

def q(query, record=None):
    if any(substring in query for substring in ['UPDATE', 'INSERT', 'DELETE']):
        return db.run_insert_query(query, record)
    elif 'SELECT' in query:
        return pd.DataFrame(db.run_select_query(query, record, column_names=True))
    else:
        print('You dont have UPDATE, INSERT, DELETE or SELECT in your query my guy')

def display_md(df):
    print(df.to_markdown())

In [51]:
project_name = 'TESTED Data Collection'
patient_name = '11001'
crf_name = 'ct'

project_name = 'CAPTIVA Data Collection'
patient_name = '9996'
crf_name = 'baseline_brain_crf'

# project_name = 'CAPTIVA MRI DATA COLLECTION'
# patient_name = '1001'
# crf_name = 'safety_reads_crf'

project_name = '29423 Vagal - SISTER'
patient_name = '101'
crf_name = 'baseline_ctctactp'
instance = None

# project_name = '14102 Khandwala-Radiology Imaging Services Core Lab Workflow'
# patient_name = '9995'
# crf_name = 'test_test'
# crf_name = 'repeat_instrument_form'


project = AMBRA_Backups.redcap_funcs.get_redcap_project(project_name)


def export_records_wrapper(patient_name, crf_name, instance=None):
    """
    wrapper is necessary because of a export_record bug. If a repeating instance form is 
    the first form in the project, a residual row is returned for other forms. This function excludes
    that residual.
    Also included an instance parameter 
    """

    form_df = pd.DataFrame(project.export_records(records=[patient_name], forms=[crf_name]))
    form_df = form_df[form_df[crf_name+'_complete'] != '']
    if instance:
        if 'redcap_repeat_instrument' not in form_df.columns:
            raise ValueError(f'''Project '{project.export_project_info()['project_title']}' does not have repeat instances.
                               \npatient_name: {patient_name}, crf_name: {crf_name}''')
        if instance not in form_df['redcap_repeat_instance'].to_list():
            raise ValueError(f'''Instance: {instance} not of available instances: {form_df['redcap_repeat_instance'].to_list()}
                               \nIn project: {project.export_project_info()['project_title']}, crf_name: {crf_name}, patient_name: {patient_name}''') 
        form_df = form_df[form_df['redcap_repeat_instance'] == instance]
    return form_df

export_records_wrapper(patient_name, crf_name, instance)

,record_id,q1001___1,q1001___2,q1001___3,q1001___4,q1002,q1003,q1004,q1005,q1006,...,q1115,q1116,q1117,q1118,q1119,q1120,baseline_ctctactp_status,baseline_ctctactp_comments,baseline_ctctactp_init,baseline_ctctactp_complete
0,101,1,1,1,1,Pre-Inclusion CTP,2024-05-02,16:07,1,1,...,1,3,1,1,Max Wintermark,No Relative CBF < 40% volume,1,There is a Pre-inclusion CTP done in addition ...,,2


In [48]:
form_df = pd.DataFrame(project.export_records(records=[patient_name], forms=[crf_name]))
# form_df = form_df[form_df['redcap_repeat_instrument'] == 1]
form_df['redcap_repeat_instrument']

0    repeat_instrument_form
Name: redcap_repeat_instrument, dtype: object